In [2]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import itertools

In [3]:
# all images will be converted to this size
ROWS = 256
COLS = 256
CHANNELS = 3

In [4]:
train_image_generator = ImageDataGenerator(horizontal_flip=True, rescale=1./255, rotation_range=45)
test_image_generator = ImageDataGenerator(horizontal_flip=False, rescale=1./255, rotation_range=0)

train_generator = train_image_generator.flow_from_directory('trains', target_size=(ROWS, COLS), class_mode='categorical')
test_generator = test_image_generator.flow_from_directory('tests', target_size=(ROWS, COLS), class_mode='categorical')

Found 11788 images belonging to 200 classes.
Found 11788 images belonging to 200 classes.


In [6]:
train_generator.reset()
test_generator.reset()

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=(ROWS, COLS, CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(200))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 254, 254, 64)      1792      
                                                                 
 activation_6 (Activation)   (None, 254, 254, 64)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 252, 252, 64)      36928     
                                                                 
 activation_7 (Activation)   (None, 252, 252, 64)      0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 63, 63, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 61, 61, 64)        36928     
                                                      

In [8]:
tensorboard = TensorBoard(log_dir='./logs/custom')

model.fit(train_generator, steps_per_epoch=512, epochs=10, callbacks=[tensorboard], verbose=2)

Epoch 1/10
512/512 - 2285s - loss: 5.1225 - accuracy: 0.0159 - 2285s/epoch - 4s/step


In [ ]:
print(model.evaluate_generator(test_generator, steps=1000))